In [1]:
import numpy as np 
import pandas as pd 
from google.cloud import bigquery
from pandas.io import gbq

In [2]:
#How many visitors went to the store? 
store_visitors="""
SELECT 
date,
COUNT(CASE WHEN hits.page.pagePath = "/store.html" THEN 1 ELSE 0 END) AS store_visitors
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
,UNNEST (hits) AS hits
WHERE _TABLE_SUFFIX BETWEEN '20160801' AND '20170731'
AND  hits.page.pagePath ="/store.html"
GROUP BY date 
ORDER BY date 
"""
store_visitors_ = gbq.read_gbq(store_visitors, project_id = 'my-project-97639')
store_visitors_.to_csv('google_store_visitors.csv', index = False)

Downloading: 100%|██████████| 365/365 [00:00<00:00, 1633.19rows/s]


In [3]:
returning_visitor="""
SELECT
date,
total_visitors,
returning_visitors,
--All about RVR https://contently.com/2015/08/18/how-loyal-are-your-customers-this-metric-has-the-answer/
((SUM(returning_visitors)/SUM(total_visitors))*100) AS RVR
FROM(SELECT
date,
COUNT(CASE WHEN visitNumber > 1 THEN fullvisitorId ELSE NULL END) AS returning_visitors,
COUNT(fullvisitorId) AS total_visitors
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE _TABLE_SUFFIX BETWEEN '20160801' AND '20170731'
GROUP BY date
ORDER BY date)
GROUP BY date, total_visitors, returning_visitors
ORDER BY date
"""
returning_visitor_ = gbq.read_gbq(returning_visitor, project_id = 'my-project-97639')
returning_visitor_.to_csv('google_returning_visitors.csv', index = False)

Downloading: 100%|██████████| 365/365 [00:00<00:00, 1603.22rows/s]


In [5]:
bounce_pageviews = """
WITH bounce_pageviews AS 
                     (SELECT 
                     COUNT(fullvisitorId)AS pageviews, 
                     SUM(totals.bounces) AS bounces,
                     hits.page.pagePath AS page
                     FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
                    ,UNNEST(hits) as hits
                    WHERE  _table_suffix BETWEEN '20160801'
                    AND '20170731'
                    AND hits.type = 'PAGE'
                    GROUP BY page                        )
                                            
    SELECT 
    pageviews, 
    page,
    IFNULL(((bounces / pageviews)*100),0) AS bounce_rate
    FROM bounce_pageviews
    GROUP BY pageviews, page, bounces
    ORDER BY PAGEVIEWS DESC
    LIMIT 20
    """
bounce_ = gbq.read_gbq(bounce_pageviews, project_id = 'my-project-97639')
bounce_.to_csv('google_bounce_pageviews.csv', index = False)

Downloading: 100%|██████████| 20/20 [00:00<00:00, 98.72rows/s]


In [6]:
visited_pages = """
 SELECT
COUNT(fullVisitorId) AS users,
SUM(visitNumber) AS visit_number,
hits.page.pagePath AS pagePath
FROM
`bigquery-public-data.google_analytics_sample.ga_sessions_*`,
UNNEST(hits) as hits
WHERE
_TABLE_SUFFIX BETWEEN '20160801' AND '20170731'
AND
--calculating data only when customer hit another page
hits.type="PAGE"
GROUP BY
pagePath
ORDER BY visit_number DESC
LIMIT 10 
"""
visited_pages_ = gbq.read_gbq(visited_pages, project_id = 'my-project-97639')
visited_pages_.to_csv('google_visited_pages.csv', index = False)

Downloading: 100%|██████████| 10/10 [00:00<00:00, 53.14rows/s]
